# Predicción de la producción de activos industriales utilizando el algoritmo de detección de anomalías Elliptic Envelope

### 1. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from itertools import combinations
import time

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import mean_squared_error


### 2. Lectura de datos y preparación

In [2]:
Y_train=pd.read_csv(('Datos/train_y.csv'))
Y_train.head()

,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,3850220.0,9019860.320
1,2,3793240.0,8568867.304
2,3,3870570.0,8549258.912
3,4,3801380.0,8451216.952
4,5,3752540.0,8323762.404


In [3]:
data=pd.read_csv('Output/X_train_formateado.csv')  #X_train_formateado.csv se obtiene mediante el código 
                                                   #del notebook 0.Exploración y preparación de datos
data.head(3)

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3


In [4]:
#Leer datos de capacidad nominal por asset por semana
capacity=pd.read_csv('Datos/assets.csv', header=0, names=['asset_id', 'nominalc_week'],
                     dtype={'asset_id':int, 'nominalc_week':float} )
#agregar columna de capacidad por día
capacity['nominalc_day']=capacity['nominalc_week']/7     
capacity.head()

,asset_id,nominalc_week,nominalc_day
0,1,132000.0,18857.142857
1,2,93000.0,13285.714286
2,3,238600.0,34085.714286
3,4,206000.0,29428.571429
4,5,310000.0,44285.714286


In [5]:
data=data.merge(capacity[['asset_id','nominalc_day']])
data

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day,nominalc_day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1,18857.142857
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2,18857.142857
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3,18857.142857
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4,18857.142857
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5,18857.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60419,83,3,104,3,1.0,0.0,48.0,8.0,4,104_3,2019-12-25,724,20857.142857
60420,83,3,104,4,0.0,0.0,37.0,1.0,4,104_4,2019-12-26,725,20857.142857
60421,83,3,104,5,1.0,0.0,48.0,9.0,4,104_5,2019-12-27,726,20857.142857
60422,83,3,104,6,2.0,0.0,53.0,11.0,4,104_6,2019-12-28,727,20857.142857


### 3. Definición de funciones

In [6]:
def ClasificacionDeObservaciones(df):
    '''FUNCIÓN PARA CLASIFICAR ANOMALÍAS MEDIANTE ELLIPTICENVELOP CONSIDERANDO t3 y t4 PARA CADA ASSET
    CONSIDERANDO CADA UNA DE LAS MEDICIONES QUE TIENE CADA ASSET
    df: pandas data frame
    ''' 
    if df.total_t.mean() == 2:
        mediciones=[3,4]
    else:
        mediciones=[1,2,3,4]
    for x in mediciones:  #para cada una de las mediciones
        t='t'+str(x)
        if df[t].unique().sum()!=0:
            cov=EllipticEnvelope(random_state=0).fit(df[['day',t]])  #contamination=0.25 para que sea la detección mediante 
                                                                     #Covarianza Robusta (Minimum Covariance Determinante)


            df['outlier_'+t]=-(cov.predict(df[['day',t]]))
            df['outlier_'+t]=np.where(df['outlier_'+t]==-1,0,df['outlier_'+t])
        continue        
    return  df

In [12]:
def resultados(dict_df):
    '''FUNCION PARA CALCULAR EL ERROR ASIGNANDO EL VALOR DE PRODUCCION NOMINAL DIARIO A LOS DÍAS EN QUE NO HUBO 
    NINGUNA ANOMALÍAS Y 0 EN CASO CONTRARIO 
    
    dict_df: diccionario resultante con la detección de anomalías con un dataframe por cada asset

    '''
    
    X=pd.concat(dict_df.values())
    X=X.reset_index(drop=True)
    
    X=X[['asset_id', 'group_id', 'week', 'weekday','total_t','nominalc_day', 'outlier_t1','outlier_t2','outlier_t3','outlier_t4']]
    

    X['produccion']=np.where(
        ( ( X['outlier_t1']==1 ) | ( X['outlier_t2']==1 ) | (X['outlier_t3']==1)  | ( X['outlier_t4']==1 ) ),
        0, X['nominalc_day'] )
   
    #Agregamos los datos por grupo y semana para obtener Y_predicted

    ProduccionGrupoSemana=X.groupby(['group_id','week'])
    Y_predicted=ProduccionGrupoSemana.agg({'produccion':'sum'})
    Y_predicted=Y_predicted.reset_index()

    # Configuramos el formato  Y_predicted para tener el mismo que en Y_train
    Y_predicted=pd.pivot_table(Y_predicted,index=['week'], columns=['group_id'], values=['produccion'])
    Y_predicted=Y_predicted.reset_index()
    Y_predicted.columns.set_levels(['production_group_2', 'production_group_3', ''], level=1, inplace=True)
    Y_predicted.columns=list(map("".join, Y_predicted))
    Y_predicted.columns=Y_train.columns

    # Calculamos el error para cada guropo
    MSE2=mean_squared_error(Y_train.PRODUCTION_GROUP_2, Y_predicted.PRODUCTION_GROUP_2)
    MSE3=mean_squared_error(Y_train.PRODUCTION_GROUP_3, Y_predicted.PRODUCTION_GROUP_3)

    
    #Calculamos el error total
    Y_train_tot = Y_train.PRODUCTION_GROUP_2.to_list() + Y_train.PRODUCTION_GROUP_3.to_list()
    Y_predicted_tot = Y_predicted.PRODUCTION_GROUP_2.to_list() +  Y_predicted.PRODUCTION_GROUP_3.to_list() 
    ErrorTotal = mean_squared_error(Y_train_tot, Y_predicted_tot)
    

    return Y_predicted, ErrorTotal

### 4. Clasificar observaciones, identificando anomalías para cada uno de las series de tiempo de los assets

In [8]:
#Crear objeto GroupBy de Pandas para iterar despues en las tablas de cada asset
data=data.set_index('asset_id')
porAsset=data.groupby(data.index)

In [9]:
start_time=time.time()
# 1 min 43 seg 
#Pasamos el subconjunto de datos de cada asset por la función de deteccion de anomalías 
#y guardamos cada uno de los dataframes resultantes con la clasificacione de las observaciones en el diccionario new_data

new_data={}
for ass in range(1,84):
    asset = porAsset.get_group(ass).reset_index()
    new_data[ass] = ClasificacionDeObservaciones(asset)
print("--- %s seconds ---"%(time.time()-start_time))

--- 114.34617376327515 seconds ---


##### Guardamos la tabla resultante en archivo csv

In [10]:
X=pd.concat(new_data.values())
X.to_csv('Output/df_outliers_EllipticEnvellope.csv',index=False)

### 5. Resultados

In [13]:
y_predicted, error= resultados(new_data)
print('error =', np.format_float_scientific(error))
y_predicted

error = 1.4464483983595354e+12


,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,2.581509e+06,5.915131e+06
1,2,2.628451e+06,5.493301e+06
2,3,3.186309e+06,4.524934e+06
3,4,3.216666e+06,4.687539e+06
4,5,3.625894e+06,5.475323e+06
...,...,...,...
99,100,2.721829e+06,6.708032e+06
100,101,2.400000e+06,5.482621e+06
101,102,2.588362e+06,4.979328e+06
102,103,2.560291e+06,5.220130e+06
